In [ ]:
import tensorflow as tf
# checkpoint_dir = '~/research/quora/quora_train/1491988932/checkpoints/'
# checkpoint_file= tf.train.latest_checkpoint(checkpoint_dir)
# print("checkpoint file: {}".format(checkpoint_file))

In [ ]:
import pandas as pd
df_test1 = pd.read_csv('test.csv')

In [ ]:
import numpy as np
import pickle
output = open('vacabulary_quora_adab.pkl', 'r')
vocabulary = pickle.load(output)
output.close()
output = open('vocabulary_inv_quora_adab.pkl', 'r') 
vocabulary_inv = pickle.load(output)
output.close()

In [ ]:
import re
def nltk_tokenize(s):    
    s = s.lower().strip()    
    return nltk.word_tokenize(s)
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.,!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s.split(' ')

def get_pairs(df_text,test_with_label= False):
    pairs_test = []
    if test_with_label:
        for q1,q2,is_dup in zip(df_text.question1,df_text.question2,df_text.is_duplicate):
            if type(q1) ==float:
                q1 = "<mino/>"
            try:
                pair1 =  nltk_tokenize(q1)
            except:
                pair1 =  normalizeString(q1)
            if type(q2) ==float:
                q2 = "<mino/>"
            try:
                pair2 =  nltk_tokenize(q2)
            except:
                pair2 =  normalizeString(q2)
            pairs_test.append([pair1,pair2,is_dup])
    else:
        for q1,q2 in zip(df_text.question1,df_text.question2):
            if type(q1) ==float:
                q1 = "<mino/>"
            try:
                pair1 =  nltk_tokenize(q1)
            except:
                pair1 =  normalizeString(q1)
            if type(q2) ==float:
                q2 = "<mino/>"
            try:
                pair2 =  nltk_tokenize(q2)
            except:
                pair2 =  normalizeString(q2)
            pairs_test.append([pair1,pair2])
    return pairs_test

In [ ]:
pairs_test1 = get_pairs(df_test1,True)

In [ ]:
len(pairs_test1)

In [ ]:
def build_data(input_data, vocabulary,training = True):
    mino_idx = vocabulary["<mino/>"]
    data_index = []
    for pair in input_data:
        sentence1 = [vocabulary.get(word, mino_idx) for word in pair[0]]
        sentence2 = [vocabulary.get(word, mino_idx) for word in pair[1]]
        if training:
            data_index.append([sentence1,sentence2,pair[2]])
        else:
            data_index.append([sentence1,sentence2])
    return data_index

In [ ]:
def get_batch(batch,training = True):
    length1 = [len(sen1) for sen1 in batch[:,0]]
    max_length1 = max(length1)
    length2 = [len(sen2) for sen2 in batch[:,1]]
    max_length2 = max(length2)
    s1_batch = []
    s2_batch = []
    for sen in batch:
        if len(sen[0])==max_length1:
            s1_batch.append(sen[0])
        else:
            s1_batch.append(sen[0]+(max_length1-len(sen[0]))*[0])
        if len(sen[1])==max_length2:
            s2_batch.append(sen[1])
        else:
            s2_batch.append(sen[1]+(max_length2-len(sen[1]))*[0])
    if training:   
        return np.array(s1_batch),np.array(s2_batch),batch[:,2],length1,length2
    else:
        return np.array(s1_batch),np.array(s2_batch),length1,length2

In [ ]:
batch_t_size = 512
test_index = build_data(pairs_test1,vocabulary,training =True)
pad_num = (len(test_index)/batch_t_size+1)*batch_t_size-len(test_index)
for i in range(pad_num):
    test_index.append([[0,0],[0,0],1])

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
checkpoint_dir = '~/research/quora/quora_train/1493868214/checkpoints/'

In [ ]:
import os
os.environ.setdefault('CUDA_VISIBLE_DEVICES','0')
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
best_loss = 10
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        ckpt_list = ckpt.all_model_checkpoint_paths
        for file_num in range(len(ckpt_list)):
            file_name = ckpt_list[file_num]
            saver = tf.train.import_meta_graph("{}.meta".format(file_name))
            saver.restore(sess, file_name)
            # Get the placeholders from the graph by name
            input_t_s1 = graph.get_operation_by_name("input_placeholder_x1").outputs[0]
            input_t_s2 = graph.get_operation_by_name("input_placeholder_x2").outputs[0]
            lengtht_1 = graph.get_operation_by_name("length1").outputs[0]
            lengtht_2 = graph.get_operation_by_name("length2").outputs[0]
            diff_length = graph.get_operation_by_name("diff_length").outputs[0]
            predictions = graph.get_operation_by_name("m3_prediction/m3_out").outputs[0]
            pred_d =[]
            true_labels = []
            for j in range(len(test_index)/batch_t_size):
                batch_s1_d,batch_s2_d,batch_labels,length1_d, length2_d= get_batch(np.array(test_index[j*batch_t_size:(j+1)*batch_t_size]),training =True)
                diff_length_Bat = np.abs(np.subtract(length1_d,length2_d))
                diff_length_Bat[diff_length_Bat>79]=79
                feed_dict = {
                    input_t_s1: batch_s1_d,
                    input_t_s2: batch_s2_d,
                    lengtht_1:length1_d,
                    lengtht_2:length2_d,
                    diff_length: diff_length_Bat
                }
                pd = sess.run(predictions,feed_dict)
                true_labels.extend(batch_labels)
                pred_d.extend(pd[:,1].astype(float))
                if j%400 ==0:
                    print('step: '+str(j))
            current_loss = log_loss(true_labels[:-pad_num], pred_d[:-pad_num])
            print('loss: '+str(current_loss)+'file: '+ file_name)
            if current_loss < best_loss:
                best_loss= current_loss
                bestfile = file_name

In [ ]:
import pickle
with open('models_quora.pkl', 'r') as f:
    ensemfiles = pickle.load(f)

In [ ]:
import os
os.environ.setdefault('CUDA_VISIBLE_DEVICES','0')
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
best_loss = 10
nums_correct = []
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        for file_name in ensemfiles[1:]:
            #file_name = ckpt_list[file_num]
            saver = tf.train.import_meta_graph("{}.meta".format(file_name))
            saver.restore(sess, file_name)
            # Get the placeholders from the graph by name
            input_t_s1 = graph.get_operation_by_name("input_placeholder_x1").outputs[0]
            input_t_s2 = graph.get_operation_by_name("input_placeholder_x2").outputs[0]
            lengtht_1 = graph.get_operation_by_name("length1").outputs[0]
            lengtht_2 = graph.get_operation_by_name("length2").outputs[0]
            diff_length = graph.get_operation_by_name("diff_length").outputs[0]
            predictions = graph.get_operation_by_name("m3_prediction/m3_out").outputs[0]
            pred_d =[]
            true_labels = []
            predprob = []
            for j in range(len(test_index)/batch_t_size):
                batch_s1_d,batch_s2_d,batch_labels,length1_d, length2_d= get_batch(np.array(test_index[j*batch_t_size:(j+1)*batch_t_size]),training =True)
                diff_length_Bat = np.abs(np.subtract(length1_d,length2_d))
                diff_length_Bat[diff_length_Bat>79]=79
                feed_dict = {
                    input_t_s1: batch_s1_d,
                    input_t_s2: batch_s2_d,
                    lengtht_1:length1_d,
                    lengtht_2:length2_d,
                    diff_length: diff_length_Bat
                }
                pd = sess.run(predictions,feed_dict)
                true_labels.extend(batch_labels)
                pred_d.extend(pd[:,1].astype(float))
                predprob.extend(pd)
            nums_correct.append(np.sum(np.equal(true_labels[:-pad_num],np.argmax(predprob[:-pad_num],axis=1)).astype(float)))   
            current_loss = log_loss(true_labels[:-pad_num], pred_d[:-pad_num])
            print('loss: '+str(current_loss)+'file: '+ file_name)
            if current_loss < best_loss:
                best_loss= current_loss
                bestfile = file_name

In [ ]:
alphas = nums_correct/sum(nums_correct)

In [ ]:
pred_pro_all = np.zeros(len(test_index))
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        for num_f,file_name in enumerate(ensemfiles[1:]):
            saver = tf.train.import_meta_graph("{}.meta".format(file_name))
            saver.restore(sess, file_name)
            # Get the placeholders from the graph by name
            input_t_s1 = graph.get_operation_by_name("input_placeholder_x1").outputs[0]
            input_t_s2 = graph.get_operation_by_name("input_placeholder_x2").outputs[0]
            lengtht_1 = graph.get_operation_by_name("length1").outputs[0]
            lengtht_2 = graph.get_operation_by_name("length2").outputs[0]
            diff_length = graph.get_operation_by_name("diff_length").outputs[0]
            predictions = graph.get_operation_by_name("m3_prediction/m3_out").outputs[0]
            pred_d =[]
            for j in range(len(test_index)/batch_t_size):
                batch_s1_d,batch_s2_d,length1_d,length2_d= get_batch(np.array(test_index[j*batch_t_size:(j+1)*batch_t_size]),training =False)
                diff_length_Bat = np.abs(np.subtract(length1_d,length2_d))
                diff_length_Bat[diff_length_Bat>79]=79
                feed_dict = {
                    input_t_s1: batch_s1_d,
                    input_t_s2: batch_s2_d,
                    lengtht_1:length1_d,
                    lengtht_2:length2_d,
                    diff_length: diff_length_Bat
                }
                pd = sess.run(predictions,feed_dict)
                pred_d.extend(pd[:,1])
                if j%400 ==0:
                    print('file: '+ str(num_f) + '  step: '+str(j))
            pred_pro_all = np.dot(pred_d,alphas[num_f])+pred_pro_all

In [ ]:
log_loss(true_labels[:4000], pred_pro_all[:4000])

In [ ]:
len(pred_pro_all)

In [ ]:
import pandas as pd
df_test_final = pd.read_csv('test_final.csv')

In [ ]:
batch_t_size = 512
pairs_test_final = get_pairs(df_test_final,False)
test_final_index = build_data(pairs_test_final,vocabulary,training =False)
pad_num = (len(test_final_index)/batch_t_size+1)*batch_t_size-len(test_final_index)
for i in range(pad_num):
    test_final_index.append([[0,0],[0,0]])

In [ ]:
#bestfile = '/home/zhangjinbin/research/quora/quora_train/1492012667/checkpoints/model-11844'
pred_pro_all = np.zeros(len(test_final_index))
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        for num_f,file_name in enumerate(ensemfiles[1:]):
            saver = tf.train.import_meta_graph("{}.meta".format(file_name))
            saver.restore(sess, file_name)
            # Get the placeholders from the graph by name
            input_t_s1 = graph.get_operation_by_name("input_placeholder_x1").outputs[0]
            input_t_s2 = graph.get_operation_by_name("input_placeholder_x2").outputs[0]
            lengtht_1 = graph.get_operation_by_name("length1").outputs[0]
            lengtht_2 = graph.get_operation_by_name("length2").outputs[0]
            diff_length = graph.get_operation_by_name("diff_length").outputs[0]
            predictions = graph.get_operation_by_name("m3_prediction/m3_out").outputs[0]
            pred_d =[]
            for j in range(len(test_final_index)/batch_t_size):
                batch_s1_d,batch_s2_d,length1_d,length2_d= get_batch(np.array(test_final_index[j*batch_t_size:(j+1)*batch_t_size]),training =False)
                diff_length_Bat = np.abs(np.subtract(length1_d,length2_d))
                diff_length_Bat[diff_length_Bat>79]=79
                feed_dict = {
                    input_t_s1: batch_s1_d,
                    input_t_s2: batch_s2_d,
                    lengtht_1:length1_d,
                    lengtht_2:length2_d,
                    diff_length: diff_length_Bat
                }
                pd = sess.run(predictions,feed_dict)
                pred_d.extend(pd[:,1].astype(float))
                if j%400 ==0:
                    print('file: '+ str(num_f) + '  step: '+str(j))
            pred_d = map(lambda x: (a*x/(a*x + b*(1-x))),pred_d)
            pred_pro_all = np.dot(pred_d,alphas[num_f])+pred_pro_all